In [ ]:
import openai
import pandas as pd
import random

In [ ]:
try:
  api_key = "INSERT KEY" # Add API key
  openai.api_key = api_key
  print("API key successfully loaded.")
except:
  print("No API key file found.")

In [ ]:
# CSV path (change accordingly)
csv_path = "lungu_scale_test.csv" 
stimuli_list = []
fields = ['Instructions', 'Condition', 'Study', 'Type', 'Proposition', 'NAND', 'NOR', 'Positive AND']  # Columns used in CSV
row_order = 1
 
# Reading CSV to dataframe
stimuli_df = pd.read_csv(csv_path, usecols=fields)
for _, row in stimuli_df.iterrows():
    stimulis = {}

    if (row['Condition'] == "Control"):
        text_parts = [row['Instructions'], row['Proposition']]
    
    else:
        # Getting continuations 
        conts = {"NAND":row["NAND"], "NOR":row["NOR"], "Contradictory":row["Positive AND"]}

        # Creating stimuli sequences
        for key, value in conts.items():
            text_parts = [row['Proposition'], value]
            stimulis[key] = ' '.join(filter(None, text_parts))

    # Other info about stimulus
    stimulis["condition"] = row['Condition']
    stimulis["study"] = row['Study']
    stimulis["type"] = row['Type']
    stimulis["order"] = row_order
    row_order += 1
    
    stimuli_list.append(stimulis)

random.shuffle(stimuli_list)

print(f"Found {len(stimuli_list)} sets of stimuli in the given CSV file.")

In [ ]:
outputs = []
NAND_lp = []
NOR_lp = []

def get_log_prob(sequence, model="gpt-3.5-turbo"):
    response = openai.Completion.create(
        model=model,
        prompt=sequence,
        max_tokens=0,
        logprobs=1,
        echo=True
    )
    log_probs = [choice['logprobs']['token_logprobs'] for choice in response['choices']]
    return sum(log_probs[0])

# Scoring and comparing each set of sequences for stimuli in stimuli list
for stimuli in stimuli_list:
    print(stimuli['NAND'])
    print(stimuli['NOR'])

    sequences = [stimuli['NAND'], stimuli['NOR']]

    # Calculate log-probabilities for each sequence
    scores = [get_log_prob(seq) for seq in sequences]

    score1, score2 = scores  # Extract scores for each sequence

    print(f"NAND: {score1} NOR: {score2}")
    NAND_lp.append(score1)
    NOR_lp.append(score2)

    if score1 > score2:
        print("First sequence makes more sense.")
        outputs.append('NAND')
    else:
        print("Second sequence makes more sense.")
        outputs.append('NOR')